# Processamento de Linguagem Natural (NLP)

Professor: Arlindo Galvão

Data: 03/09/2024

## Cronograma

* Parte I: N-Gram Language Model

* Parte II: Character-Level RNN Language Model

## OBS
Deixar registrado as repostas nas saídas das celulas do notebook de submissão.

## Parte I: N-Gram Language Model

In [1]:
'''Imports'''

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import unicodedata
import re
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import random
from nltk.corpus import stopwords

In [1262]:
'''Classe N-gram
    O modelo N-Gram consiste em utilizar os n últimos tokens para prever o próximo token, realizando uma espécie de janela deslizante no texto.
    O modelo recebe um token em sua representação numérica e utiliza embeddings para converter essa representação em vetores densos situados em um espaço vetorial.
    Esses vetores são passados para uma camada linear que os processa e gera uma saída de tamanho 128, que será passada na rede para gerar a distribuição de scores sobre cada token do vocabulário. (utilizando a LogSoftmax)
    
    Vale ressaltar que esse modelo possui entrada fixa, como também não faz um processamento temporal da entrada.
'''

class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # Gera uma matriz com vocab_size linhas e embedding_dim colunas, onde cada token é associado a um índice e nesse indíce está os embeddings associados aquele token, no inicio, aleatórios
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)   # FC com 128 neurônios, casa um com um peso para cada dimensão de cada token de entrada
        self.linear2 = nn.Linear(128, vocab_size) # FC com vocab_size neurônios, pois cada neurônio é responsável por gerar uma probabilidade para cada token do vocabulário

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1)) # Gera embeddings para os tokens de entrada e achata eles para passar para a camada linear
        out = F.relu(self.linear1(embeds)) # Aplica a camada linear e a função de ativação ReLU
        out = self.linear2(out) # Aplica a segunda camada linear, reponsável por gerar as probabilidades
        log_probs = F.log_softmax(out, dim=1) # Aplica a função de ativação softmax para gerar uma distribuição de probabilidade
        return log_probs

In [1263]:
# Parâmetros do modelo
'''O context_size define quantos grams passaremos para o modelo, ou seja, quantos tokens passados serão utilizados para prever o próximo token.
Já o embedding_dim define quantas posições o vetor de representação das palavras possuirá, assim, a palavra/token passa a ser representada como um vetor com embedding_dim dimensões.'''

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [1264]:
# Criar n-grams
ngrams = [([test_sentence[i - CONTEXT_SIZE + j] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]

# Construir o vocabulário
vocab = set(test_sentence) # A priori, cada token está sendo considerado como uma palavra
word_to_ix = {word: i for i, word in enumerate(vocab)} # Cada palavra recebe seu índice, e esses índices servem para localizar os embeddings desa palavra
ix_to_word = {i: word for word, i in word_to_ix.items()}

In [1265]:
# Treinamento

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams: # Para cada n-grama 
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

    print(f'{epoch} - Loss: {total_loss}')


#print("Losses:", losses)

### Atividades

__1 - Escreva a função generate da classe NGramLanguageModel.__

__2 - Depois de treinar o modelo, gere uma sentença de 128 tokens.__

__3 - Calcule e print a similaridade entre duas palavras. A similaridade resultante está correta? Justifique a sua resposta.__

__4 - Proponha três alterações no código e demonstre que melhorou o desempenho do modelo.__

####  Atividade 1

REQUISITOS:
Para gerar 128 tokens é necessário:
- ter um contexto para passar para o modelo
- realizar o foward no modelo já treinado com esse contexto inicial
- escolher o token com maior probabilidade e adicionar ao contexto
- pegar os ultimos n grams do contexto e passar para o modelo (etapa 1)


In [1266]:
'''O código da função generate segue a seguinte lógica:
    - Recebe o modelo, o contexto inicial (necessário para a primeira predição) e o número de palavras/tokens a serem gerados
    
    - Para cada palavra que deseja gerar:
        - Transforma o contexto em índices para acessar seus respectivos embeddings
        - Com o modelo treinado, retorna as probabilidades de cada token/palavra do vocabulário
        - Escolhe a palavra de maior probabilidade
        - Adiciona a nova palavra prevista ao contexto
        - Retira o primeiro elemento do contexto para dar continuidade ao texto com o mesmo tamanho de entrada
        - Printa a palavra prevista
'''

def generate(model, contexto, n_palavras):
    for i in range(n_palavras): # Para cada palavra que queremos gerar
        context_idxs = torch.tensor([word_to_ix[w] for w in contexto], dtype=torch.long) # Transforma o contexto em índices para acessar seus respectivos embeddings

        with torch.no_grad(): # Utilizando torch.no_grad() para não armazenar os gradientes 
            log_probs = model(context_idxs) # Retorna as probabilidades de cada token/palavra do vocabulário, uma estratégia poderia ser fazer um top-k, onde pega-se as k palavras com maior probabilidade e escolhe-se uma aleatória, dando uma estocasticidade ao modelo.

        palavra_predita_id = torch.argmax(log_probs).item() # Escolhe-se a palavra de maior probabilidade
        palavra_predita = ix_to_word[palavra_predita_id] # Acessa-se a palavra associada ao índice

        contexto.pop(0) # Retira o primeiro elemento do contexto para adicionar a nova palavra prevista e dar continuidade ao texto
        contexto.append(palavra_predita)

        print(palavra_predita, end=' ')


In [1267]:
contexto = ['When', 'forty']
n_palavras = 128

generate(model, contexto, n_palavras)

#### Atividade 3

Para calcular a similaridade entre duas palavras, será utilizado similaridade de cossenos. Com isso, enxergamos os tokens(palavras) como vetores em um espaço vetorial com a hipótese de que as distâncias preservam informação semântica (não necessariamente verdade), visto que utilizamos palavras vizinhas para prever a próxima palavra, assim, a representação (embedding) leva em consideração a semântica, visto que duas palavras que aparexem em contextos similares tendem a ter representações similares.

Para isso, precisamos:
- Escolher duas palavras
- Obter os embeddings das palavras (representação vetorial)
- Calcular a similaridade de cossenos entre os vetores através da formula:
- $cos(\theta) = \frac{A \cdot B}{||A|| \cdot ||B||}$

A similaridade de cossenos nos retorna um valor entre -1 e 1, onde 1 indica que os vetores "apontam" na mesma direção dentro do espaço e -1 indica que são opostos e 0 indica que são ortogonais.

A medida de similaridade de cossenos nos traz a informação sobre a direção do vetor dentro do espaço, ignorando a sua magnitude, por isso, é possível que dois vetores tenham similaridade de cossenos alta, mas distâncias euclidianas diferentes.

In [1268]:
def similaridade_cossenos(model, palavra1, palavra2):
    idx_palavra1 = torch.tensor([word_to_ix[palavra1]], dtype=torch.long) # Transforma a palavra em seu índice para acessar o embedding (representação vetorial)
    idx_palavra2 = torch.tensor([word_to_ix[palavra2]], dtype=torch.long)

    with torch.no_grad():
        v1 = model.embeddings(idx_palavra1) # Acessa o embedding da palavra, trazendo sua representação vetorial
        v2 = model.embeddings(idx_palavra2)

    produto_interno = torch.dot(v1.view(-1), v2.view(-1)) # Realiza o produto interno entre os vetores, que pode ser visto como uma distância entre os vetores no espaço

    norma_v1 = torch.norm(v1) # Calcula a norma dos vetores
    norma_v2 = torch.norm(v2)

    similaridade = produto_interno / (norma_v1 * norma_v2)


    return round(similaridade.item(), 4) # Retorna a similaridade arrendondada em 4 casas decimais

In [1269]:
palavras_teste = [['beauty', 'beauty'], ['beauty', 'field,'], ['beauty', 'shame,'], ['beauty', 'praise'], ['beauty', "beauty's"]]

for p1, p2 in palavras_teste:
    print(f'Similaridade entre {p1} e {p2}: {similaridade_cossenos(model, p1, p2)}')

Vale ressaltar que não necessariamente essas representações vetoriais preservam informação semântica ou alguma "noção" sobre as palavras, isso por que o modelo é puramente matemático, ná prática, a representação de uma palavra vai ser corrigida para minimizar uma função (loss) para aquela tarefa específica, e não necessariamente para preservar informação semântica ou fazer algum sentido para o ser humano. Ainda assim, assumimos que a têndencia é, que se treinado para uma tarefa em que dado uma palavra utilizamos seu contexto, e vice versa, provavelmente teremos uma informação semântica preservada, isso por que, se o corpus for suficientemente grande, palavras "similares" tenderão a aparecer em contextos similares, com isso, é como se repetidas vezes, ao entrar com a mesma (ou quase a mesma) informação na rede tivessemos duas ou mais palavras altamente provaveis, com isso, o natural é que ambas tenham representações vetoriais parecidas. No texto acima, o corpus não é suficientemente grande para que isso aconteça.

#### Atividade 4

**Alteração 1:** Aumentar a quantidade de ajustes do modelo. O modelo incialmente proposto realiza uma atualização de pesos a cada amostra passada, porém faz isso apenas 10 vezes. Como temos poucos dados, na prática temos também poucos ajustes, aumentar a quantidade de ajustes pode melhorar o desempenho do modelo.

**Alteração 2:** Aumentar a dimensão dos embeddings. Aumentar a dimensão dos embeddings pode ser uma estratégia para melhorar a representação semântica das palavras.

**Alteração 3:** Aumentar o contexto. Aumentar o contexto pode ser uma estratégia para melhorar a previsão da próxima palavra, visto que mais palavras podem ser utilizadas para prever a próxima palavra.

**Alteração 4:** Aumentar learning_rate

In [1270]:
num_experimentos = 4
resultados = []
contexto = ['When', 'forty']
n_palavras = 128

for experimento in range(num_experimentos):

    if experimento == 0:
        print("Experimento 1: Aumentar a quantidade de ajustes do modelo")
        epocas = 300
        print_every = 50
        learning_rate = 0.001
        CONTEXT_SIZE = 2
        EMBEDDING_DIM = 10

        ngrams = [([test_sentence[i - CONTEXT_SIZE + j] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]

        # Construir o vocabulário
        vocab = set(test_sentence)
        word_to_ix = {word: i for i, word in enumerate(vocab)}
        ix_to_word = {i: word for word, i in word_to_ix.items()}

    if experimento == 1:
        print("Experimento 2: Aumentar a dimensão dos embeddings")
        EMBEDDING_DIM = 30

    if experimento == 2:
        print("Experimento 3: Aumentar o contexto")
        CONTEXT_SIZE = 4
        contexto = ['When', 'forty', 'winters', 'shall' ]
        ngrams = [([test_sentence[i - CONTEXT_SIZE + j] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]

    if experimento == 3:
        print("Experimento 4: Aumentar learning_rate")
        learning_rate = 0.01

    print(f"Experimento {experimento + 1} de {num_experimentos}")

    model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    loss_function = nn.NLLLoss()
    losses = []

    for epoch in range(epocas):
        total_loss = 0
        for context, target in ngrams:
            context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
            model.zero_grad()
            log_probs = model(context_idxs)
            loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        losses.append(total_loss)

        if epoch % print_every == 0:
            print(f'Época: {epoch} - Loss: {total_loss}')

    resultados.append(losses)
    print()

    # Gerando texto
    print('Gerando texto: ', end=' ')
    generate(model, contexto, n_palavras)  # Corrigido passando word_to_ix e ix_to_word
    print('\n')

# Exibir os resultados ao final de todos os experimentos
for i, resultado in enumerate(resultados):
    print(f"Resultado do experimento {i + 1}: Última perda: {resultado[-1]}")


In [1271]:
"""PLOTAR GRAFICO COMPARANDO OS 4 MODELOS E LOSS AO LONGO DO TEMPO"""

def plotar_perdas(resultados):
    """
    Plota as curvas de perda para cada experimento.

    Parâmetros:
    resultados (list of lists): Lista onde cada sublista contém as perdas por época de um experimento.
    """
    plt.figure(figsize=(12, 8))  # Define o tamanho da figura

    # Define um ciclo de cores para os experimentos
    cores = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown']

    for i, perdas in enumerate(resultados):
        epocas = range(1, len(perdas) + 1)
        cor = cores[i % len(cores)]  # Cicla as cores se houver mais experimentos que cores disponíveis
        plt.plot(epocas, perdas, label=f'Experimento {i+1}', color=cor)

    plt.xlabel('Épocas', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.title('Curvas de Perda por Experimento', fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


plotar_perdas(resultados)

## Parte II: Character-Level RNN Language Model

In [2]:
"""As RNN lidam com sequências de dados, de maneira que cada item da sequencia é processado em um instante de tempo. Com isso, fazemos o processamento sequencial da entrada. O papel da RNN consiste em gerar uma boa representação da sequência em um vetor que chamamos de hidden_state, de maneira que o hidden_state é o vetor que em teoria armazena informação de todos os itens da sequência. Esse hidden_state é atualizado a cada novo item, permitindo que a rede aprenda dependências temporais. Na prática, temos duas redes dentro do bloco da rnn, uma responsável por receber o hidden_state atual e o item da sequência naquele instante de tempo e gerar uma nova representação desse hidden_state, já o outro modelo recebe esse hidden_state e realiza uma classificação, assim, é possível avaliar a qualidade da representação do hidden gerado pela primeira rede. 

Temos também que o processamento de sequências pode ser um problema, se lembrarmos das redes neurais, vemos que a partir de um certo número de camadas, começamos a enfrentar problemas com o gradiente, que pode explodir ou sumir (exploding e vanish gradient). Isso motivou o surgimento de técnicas como o batch_normalization e Ligações residuais. No processamento de sequências esse problema está ainda mais presente do que nas redes vanillas, isso por que, além de que quanto mais itens na sequência mais dependencia de camadas temos e mais fácil de termos problemas com o gradiente, ainda por que, estamos desenrolando a mesma rede ao longo do tempo, com isso, ainda existe a desvantagem das redes recorrentes de possuir um grau de liberdade a menos que as redes feedforwar."""

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

#### Funções auxiliares (Questão 2)

In [270]:
'''FUNÇÃO LOAD_DATASET'''
def load_dataset(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()


'''FUNÇÃO PREPROCESS_DATA'''
def preprocess_data(corpus):
    corpus = unicodedata.normalize('NFD', corpus)  # Remove acentuação
    corpus = re.sub(r'\s+', ' ', corpus)  # Substitui quebras de linha e espaços múltiplos por um único espaço
    corpus = corpus.lower()  # Converte para minúsculas
    corpus = re.sub(r'[^a-zA-Z0-9.,;:!?\'" -]', '', corpus)  # Remove tudo exceto letras, números, pontuações e espaços

    corpus = corpus.replace('--', '')
    return corpus


'''FUNÇÃO TOKENIZE_DATA'''
def tokenize_data(corpus):
    caracteres = list(corpus) #quebrando o texto em uma lista onde cada elemento é um caracter em ordem

    vocabulario = {caractere: i for i, caractere in enumerate(set(caracteres))} 
    tokens = [vocabulario[caractere] for caractere in caracteres]

    return tokens, vocabulario

In [11]:
test_sentence = load_dataset('obama-data.txt')

test_sentence = preprocess_data(test_sentence)
test_sentence, vocab = tokenize_data(test_sentence)

vocab_len = len(vocab)
vocab_ix_to_char = {v: k for k, v in vocab.items()}

In [271]:
'''Para a função create_input, recebemos os tokens (uma lista de números que representam caracteres), com isso, geramos um tensor de entrada que tem tamanho max_sequence_len e cada item dentro do tensor é um token representado por um one-hot-encoding. O target é um tensor do mesmo tamanho do input, onde a representação dos tokens se dá de forma númerica, pois serão vistos como classes na hora do treinamento, além disso, o target_tensor é simplesmente o tensor de input com um shift para direita, pois para cada token + hidden_state que analizarmos o target é o próximo token.'''

def create_input(tokens, start_idx, max_sequence_len):
    input = []

    t = torch.tensor(tokens[start_idx: start_idx + max_sequence_len], dtype=torch.long)

    for i in range(max_sequence_len):
        one_hot = torch.zeros(1, vocab_len)  # Inicializando o input do mesmo formato do hidden, para facilitar a concatenação dentro do modelo
        one_hot[0][t[i]] = 1  # Preenche 1 no indice do token passado (representação do token em one-hot)
        input.append(one_hot)

    target = torch.tensor(tokens[start_idx + 1: start_idx + max_sequence_len + 1], dtype=torch.long) # Cria o tensor de target, onde o mesmo é o próximo input sem a representação vetorizada

    input = torch.stack(input) # A sequencia é uma pilha com max_sequence_len tensores dentro, cada tensor é a representação daquele token 

    return input, target

In [13]:
# Parâmetros do modelo
n_hidden = 64
learning_rate = 0.001
n_epochs = 5
print_every = 1
max_sequence_len = 25

In [14]:
rnn = RNN(vocab_len, n_hidden, vocab_len)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

rnn

Primeira coisa que temos definido é que existe uma RNN que trabalha a nível de caracter, ou seja, cada caracter deve virar um token, nesse sentido, temos um vocabulário de tamanho 46. Desse modo, o tamanho da entrada é o tamanho do vocabulário + o tamanho di hidden state, isso por que cada token é representado no formato de one hot encode, onde na posição referente ao seu valor númerico como token, temos o valor 1, e 0 no restante, assim, cada token tem em sua representação o tamanho do vocabulário. 

Além disso, a rnn por si só pode ser entendido como um modelo que, dada uma sequência, gera a melhor representação possível daquela sequência em um vetor de tamanho hidden_size, onde esse vetor de hidden_size será usado para classificação, no nosso caso, do próximo token. Por isso existem duas totalmente conectadas, uma responsável por gerar a melhor representação possível, e outra por gerar a probabilidde de cada token do vocabulário ser o próximo token a partir dessa representação. O aprendizado dessa rede funciona por BTT, que possui alguns problemas conforme as sequências crescem, como o gradiente explodir ou desaparecer.

In [287]:
"""Dentro do treinamento recebemos o input e o target, e uma visão interessante é que treinamos essa rede por teaching force, isso por que ao invés de propagar para a rede, no próximo espaço de tempo, o token previsto no espaço anterior, propagamos diretamente o token correto, isso por que, sem essa técnica, seria muito dificil o modelo começar a convergir, visto que iria gerar representações erradas duranto boa parte do treino. 

O treinamento funciona da seguinte forma:
    Passasse uma sequência de max_sequence_len tokens:
        A rede gera o primeiro hidden e o primeiro output usando o x0 e o h0
        A rede usa o h1 (gerado pela rede avaliada no x0 + h0) e o x1 para gerar o h2 e o output 2.
        O processo segue até que todas entradas tenham sido processadas.
        Com isso somamos as losses e corrigimos o modelo."""

def train(input_tensor, target_tensor):

    hidden = rnn.initHidden()
    rnn.zero_grad()
    loss = 0
    for i in range(input_tensor.size(0)):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, target_tensor[i].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.item() / input_tensor.size(0)

In [16]:
fator_desconto = 10

for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')

###  Atividades

__1 - Escreva a função generate da classe RNN.__

__2 - Escreva as funções de load_dataset, preprocess_data, tokenize_data e create_input.__

__3 - Realize otimização de hiperparâmetros. Justifique a escolha dos hiperparâmetros otimizados e o espaço de busca definido.__

__4 - Adicione uma Layer de Dropout na classe RNN. Treine o novo modelo e argumente sobre o impacto dessa alteração no modelo.__

__5 - Adicione uma nova nn.Layer que recebe como input os vetores hidden e output combinados. Treine o novo modelo e argumente sobre o impacto dessa alteração no modelo.__

__6 - Adicione uma função para printar uma geração de texto de no máximo 100 caracteres sempre que printar a loss do modelo.__

__7 - Adicione uma função que calcula a perplexidade e printe com a loss.__

__8 - Proponha três alterações no código e demonstre que melhorou o desempenho do modelo.__

**Desafio**

__1 - Desenvolva um modelo Word-Level utilizando LSTM.__
- __Escrever a classe LSTM.__
- __Escrever a função generate.__
- __Otimizar o modelo.__
- __Comparar com as outras abordagens acima__

#### Atividade 1

In [18]:
"""
Ideia da função generate:
    
    Temos a entrada como uma texto em formato string, podendo conter qualquer tamanho.
    
    Préprocessamos o texto com a mesma função usada no treino e tokenizamos o texto, onde cada carácter é um token e receberá sua representação númerica.
    
    Após isso, iteramos em todos caracteres do prompt para gerar o hidden-state inicial, não faz sentido olharmos para as previsões do modelo nessa etapa, justamente por que,
    já possuimos o label, que seria o próximo token do prompt de entrada.
    
    Após pre-processar toda sequencia e obter a sua representação no vetor hidden, passamos a olhar para o output da rede. Esse output vem no formato de uma distribuição de probabilidade dado pela softmax. A softmax gera uma função densidade de probabilidade onde a soma dos valores é igual a 1 e não existe nenhum valor menor que 0, se integrarmos a softmax de -inf a +inf obteremos 1 como resposta. Para nosso modelo, estamos usando a LogSoftmax, onde retiramos o log do valor gerado pela softmax, como a softmax só produz valores entre 0 e 1, todos nossos tokens terão um valor negativo associado, para facilitar a explicação, vamos assumir que estamos usando puramente a softmax, visto que o efeito é semelhante.
    
    Com um valor referente a uma probabilidade/score de cada token, pegamos o token de maior probabilidade/score caso o aleatorio seja igual a False e o adicionamos ao prompt, repetindo esse processo, até gerar 100 tokens.
   """

def generate_rnn(rnn, prompt=' ', tamanho_resposta=100, aleatorio = True ,top_k=3):
    rnn.eval() # Modo de avaliação do modelo, desliga dropout e etc
    oculto = rnn.initHidden() # Inicialização do hidden
    prompt_processado = preprocess_data(prompt)
    prompt_tensor = torch.tensor([vocab[i] for i in list(prompt_processado)], dtype=torch.int16) # Prompt pré processado e tokenizado

    for token in prompt_tensor:
        one_hot = torch.zeros(1, len(vocab))
        one_hot[0][token] = 1 # Gerar a representação em one-hot-encoding para cada token

    for _ in range(tamanho_resposta):
        y_pred, oculto = rnn(one_hot, oculto) # Passar o token e o hidden para a rede, que devolve o novo hidden e uma distribuição de probabilidades
        if not aleatorio: # Escolhe sempre o token de maior probabilidade
            token_previsto = torch.argmax(y_pred, dim=1) # Escolhe o token previsto como o de maior probabilidade
        else: # A es
            y_pred = torch.softmax(y_pred, dim=1)
            top_k_valores, top_k_indices = torch.topk(y_pred, top_k)
            top_k_valores = top_k_valores / torch.sum(top_k_valores)
            token_previsto = top_k_indices[0, torch.multinomial(top_k_valores, 1)]

        print(vocab_ix_to_char[token_previsto.item()], end='')
        one_hot = torch.zeros(1, len(vocab))
        one_hot[0][token_previsto] = 1

In [19]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=False)

In [20]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=True)

#### BALANCEANDO SEQUÊNCIAS

In [272]:
"""É possível perceber que mesmo conseguindo gerar palavras através de caracteres temos uma repetição alta. Em outros testes, foi percebido que o modelo tende a repetir bastante palavras como the, to e etc, que são de alta frequência no texto, vamos verificar essa hipótese:"""

texto = load_dataset('obama-data.txt')
texto = preprocess_data(texto)
palavras = texto.split()
contagem_palavras = Counter(palavras)


df_contagem = pd.DataFrame(contagem_palavras.items(), columns=['Palavra', 'Frequencia'])
df_contagem = df_contagem.sort_values(by='Frequencia', ascending=False)
media_frequencia = df_contagem['Frequencia'].mean()
df_contagem_limited = df_contagem.head(20)

plt.figure(figsize=(10, 5))
plt.bar(df_contagem_limited['Palavra'], df_contagem_limited['Frequencia'], color='skyblue')
plt.axhline(y=media_frequencia, color='red', linestyle='--', label=f'Média: {media_frequencia:.2f}')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Palavras')
plt.ylabel('Frequência')
plt.title('Top 20 Palavras mais Frequentes no Texto')
plt.legend()
plt.tight_layout()

plt.show()

In [273]:
top_20 = sum(list(df_contagem_limited['Frequencia']))
todos = sum(list(df_contagem['Frequencia'])) - top_20

print('Palavras no top 20:', top_20)
print('Restante das palavras:', todos)

A linha em vermelho é a média de frequência das palavras, com isso, podemos notar que o modelo corrige muitoo mais certas palavras do que outras, o que pode prejudicar nosso treinamento. Ainda podemos ver que praticamente 1/3 das palavras estão no top 20, o que pode ser um problema para o modelo, visto que ele tende a repetir palavras de alta frequência.

In [274]:
'''A ideia é que caso uma palavra seja stopword, adicionamos 50% de chance de removê-lá do texto, caso ela permaneça nas mais frequentes'''
def augment_text_by_word_freq(text, word_counts, prob_map, stopword_prob=0.98):
    stop_words = set(stopwords.words('english'))

    palavras_prob = {}
    start = 0
    for top_n, prob in prob_map.items():
        palavras_frequentes = word_counts.most_common(top_n)[start:top_n]
        palavras_prob.update({palavra: prob for palavra, _ in palavras_frequentes})
        start = top_n

    palavras_texto = text.split()
    augmented_text = []

    for palavra in palavras_texto:
        if palavra in stop_words:
            if random.uniform(0, 1) < stopword_prob:
                continue
                
        elif palavra in palavras_prob:
            if random.uniform(0, 1) < palavras_prob[palavra]:
                continue

        augmented_text.append(palavra)

    return ' '.join(augmented_text)

In [275]:
test_sentence = load_dataset('obama-data.txt')

test_sentence = preprocess_data(test_sentence)

prob_map = {
    14: 0.5,  # Para as 100 palavras mais frequentes, 90% de chance de remoção
    20: 0.2,  # Para as próximas 100 palavras, 50% de chance de remoção
    50: 0.1   # Para as próximas 100 palavras, 20% de chance de remoção
}
test_sentence = augment_text_by_word_freq(test_sentence, contagem_palavras, prob_map, stopword_prob=0.95)

In [276]:
"""É possível perceber que mesmo conseguindo gerar palavras através de caracteres temos uma repetição alta. Em outros testes, foi percebido que o modelo tende a repetir bastante palavras como the, to e etc, que são de alta frequência no texto, vamos verificar essa hipótese:"""

palavras = test_sentence.split()

contagem_palavras = Counter(palavras)

df_contagem = pd.DataFrame(contagem_palavras.items(), columns=['Palavra', 'Frequencia'])
df_contagem = df_contagem.sort_values(by='Frequencia', ascending=False)
media_frequencia = df_contagem['Frequencia'].mean()

df_contagem_limited = df_contagem.head(20)

plt.figure(figsize=(10, 5))
plt.bar(df_contagem_limited['Palavra'], df_contagem_limited['Frequencia'], color='skyblue')
plt.axhline(y=media_frequencia, color='red', linestyle='--', label=f'Média: {media_frequencia:.2f}')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Palavras')
plt.ylabel('Frequência')
plt.title('Top 20 Palavras mais Frequentes no Texto')
plt.legend()
plt.tight_layout()

plt.show()

In [277]:
test_sentence, vocab = tokenize_data(test_sentence)

vocab_len = len(vocab)
vocab_ix_to_char = {v: k for k, v in vocab.items()}

#### Retreinando a rnn

In [60]:
fator_desconto = 10

for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')

generate_rnn(rnn, prompt='when', tamanho_resposta=100)

In [61]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=False)

In [62]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=True)

#### Atividade 3 (otimização de hiperparâmetros)

A rnn fornecida possui dois principais hiperparêmetros para otimização que são o hidden_size e o learning_rate. 

  -Hidden_size: o hidden_size é o tamanho do vetor que representa a sequência, além disso, temos duas lineares dentro da rnn, a i2h que é responsável por gerar o próximo hidden_state, ou seja, sua saída possui hidden_size valores, e portanto, são hidden_size neurônios presentes nessa camada. Com isso, quanto maior o hidden_size, mais neurônios teremos na rede e consequentemente um maior grau de liberdade do modelo, além de um maior vetor para representação do hidden_state
  
  -Já a learning_rate é o tamanho do passo que o otimizador dá para corrigir os pesos da rede, visto que na atualização de pesos do modelo cada peso w passará pela seguinte equação w = w - grad*learning_rate, assim ,um learning_rate muito pequeno pode fazer com que o modelo demore muito para convergir, enquanto um muito grande pode fazer com que o modelo não consiga convergir, ou até mesmo divergir, por sair "pulando" no espaço da função de perca de modo que nunca vá em direção a um mínimo, seja ele local ou global.
  
Para exploração dessas váriaveis temos a seguinte intuição:
  -Hidden_size: Quanto maior o hidden_size, mais informação a rede pode armazenar, porém, também mais difícil de treinar, visto que mais neurônios significa mais parâmetros a serem ajustados, além disso, um hidden_size muito grande pode fazer com que o modelo se sobreajuste aos dados, visto que terá muita capacidade de representação. Por outro lado, um hidden_size muito pequeno pode fazer com que o modelo não consiga aprender a representação da sequência, visto que não terá capacidade suficiente para armazenar informação.
    
  -Learning_rate: Quanto maior o learning_rate, mais rápido o modelo converge, porém, também mais fácil de divergir, visto que o otimizador pode "pular" o mínimo local ou global. Um learning_rate muito pequeno pode fazer com que o modelo demore muito para convergir, ou até mesmo não convergir, visto que o passo é muito pequeno, além disso, pode fazer com que o modelo fique preso em mínimos locais.

In [1213]:
'''Com isso, vamos otimizar esses hiperparâmetros, para isso, vamos definir um espaço de busca para cada hiperparâmetro e realizar uma busca em grade (grid_search), onde testaremos todas combinações possíveis de hiperparâmetros e escolheremos a melhor combinação. Os valores foram escolhidos de maneira árbritária, com base em conhecimento prévio e intuição, além disso, testaremos os hiperparâmetros em uma parcela reduzida dos dados fornecidos, devido ao alto custo computacional.'''

n_hidden_options = [32, 64, 128, 256]
learning_rate_options = [0.1, 0.01, 0.001, 0.0001]

In [805]:
n_epochs = 5
print_every = 1
max_sequence_len = 25
fator_desconto = 100

In [806]:
results = []

In [808]:
for n_hidden in n_hidden_options:
    for learning_rate in learning_rate_options:
        print(f'\nTreinando com n_hidden={n_hidden} e learning_rate={learning_rate}')

        rnn = RNN(vocab_len, n_hidden, vocab_len)
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

        total_loss = 0

        for epoch in range(1, n_epochs + 1):
            epoch_loss = 0
            for start_idx in range(0, len(test_sentence) - max_sequence_len, fator_desconto * max_sequence_len):
                input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
                loss = train(input_tensor, target_tensor)
                epoch_loss += loss

            if epoch % print_every == 0:
                print(f'Epoch: {epoch}, Loss: {epoch_loss}')
            total_loss += epoch_loss

        results.append({
            'n_hidden': n_hidden,
            'learning_rate': learning_rate,
            'total_loss': total_loss
        })
        print()

In [817]:
df_results = pd.DataFrame(results)


unique_n_hidden = df_results['n_hidden'].unique()

plt.figure(figsize=(10, 6))

for n_hidden in unique_n_hidden:
    subset = df_results[df_results['n_hidden'] == n_hidden]
    plt.plot(subset['learning_rate'], subset['total_loss'], label=f'n_hidden={n_hidden}', marker='o')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Total Loss (Log Scale)')
plt.title('Total Loss vs Learning Rate for Different n_hidden Values')
plt.legend()
plt.grid(True)

plt.show()

In [826]:
plt.figure(figsize=(10, 6))

for n_hidden in unique_n_hidden:
    subset = df_results[df_results['n_hidden'] == n_hidden]
    plt.plot(subset['learning_rate'], subset['total_loss'], label=f'n_hidden={n_hidden}', marker='o')

plt.xscale('log')
plt.yscale('log')
plt.ylim(1.5*10**4, 2.5*10**4)  # Ajuste para melhorar a visualização no início do gráfico
plt.xlabel('Learning Rate')
plt.ylabel('Total Loss (Log Scale)')
plt.title('Total Loss vs Learning Rate for Different n_hidden Values (Adjusted Y Scale)')
plt.legend()
plt.grid(True)

plt.show()

Assim a combinação de valores queresultou na menor perca foi n_hidden = 128 e learning_rate = 10**(-3)

#### Atividade 4

In [180]:
"""Adicionando dropout:
    
    Em redes neurais, existe um teorema que uma mlp, se suficientemente grande, é um aproximador universal de funções, contudo, na prática, não sabemos exatamente a função que descreve o fenômeno modelado, estimamos ela através dos dados, onde é possível ter um modelo com alta variância. Isso pode ser problemático pois indica que o modelo se sobreajustou aqueles dados,que não necessariamente descrevem a função alvo ou apresentam falta de informação, gerando um modelo muito bom, porém apenas para aquele conjunto específico que tivemos acesso.
    
    Com isso, há a utilização do dropout a fim de trazer uma regularização a rede, onde, de maneira aleatória, alguns neurônios são desligados durante o treinamento, fazendo com que a rede possua um universo de funções alcançaveis reduzido, e consequentemente 'aprenda' a não depender de um único neurônio, ou um conjunto específico deles, para fazer uma predição. Essa dificultação diminui a chance de um overfitting, e possui uma visão interessante onde podemos visualizar a técnica de dropout como um "ensemble" de redes neurais, visto que a cada foward pass, provavelmente estamos utilizando uma configuração de rede diferente, com neurônios diferentes desligados.
    
    Assim, espera-se um modelo com melhor capacidade de generalização."""

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.dropout = nn.Dropout(dropout_prob) # Adição da camada de dropout

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        
        hidden = F.tanh(hidden)

        hidden = self.dropout(hidden) # Adição do dropout
        
        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [181]:
n_hidden = 128
dropout_prob = 0.3
rnn = RNN(vocab_len, n_hidden, vocab_len, dropout_prob)

In [182]:
learning_rate = 0.001
n_epochs = 5
print_every = 1
max_sequence_len = 25
fator_desconto = 10

In [183]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

In [184]:
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')

In [185]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=False)

In [186]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100)

#### Atividade 5

In [212]:
"""A mudança cria um loop de feedback onde a saída em cada passo de tempo influencia o estado oculto para o próximo passo. Isso pode ser benéfico em alguns aspectos, por exemplo, corrigimos diretamente o erro que a camada anterior causa ao propagar um token errado para a rede no próximo instante de tempo, além disso, estamos dando um maior grau de liberdade a rede, que agora possui mais parâmetros, o que também pode ser benéfico, mas pode também causar um overfitting. Além disso, estamos adicionando um caminho no gradiente, o que pode ser problemático, visto que o gradiente pode explodir ou sumir, e com um caminho a mais, a chance disso acontecer é ainda maior."""

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.ho2h = nn.Linear(hidden_size + output_size, hidden_size) # Nova camada que recebe hidden e output concatenados

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        hidden = torch.tanh(hidden)

        output = self.i2o(hidden)
        output = self.softmax(output)

        ho_combined = torch.cat((hidden, output), 1)
        hidden = self.ho2h(ho_combined)
        hidden = torch.tanh(hidden)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


In [213]:
n_hidden = 128
rnn = RNN(vocab_len, n_hidden, vocab_len)

In [214]:
learning_rate = 0.0005
n_epochs = 5
print_every = 1

In [215]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

In [216]:
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')

In [217]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio = False)

In [218]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio = True)

Mesmo gerando palavras, não tivemos um bom resultado na geração de texto, estamos maximizando as probabilidades, mas não de maneira suficiente para que o token mais provavel seja o correto

#### Atividade 6

In [219]:
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        generate_rnn(rnn, prompt='when', tamanho_resposta=100)

#### Atividade 7

$$ \text{perplexity} = \left( \prod_{i=1}^{n} \frac{1}{P(w_i | w_1, w_2, \ldots, w_{i-1})} \right)^{\frac{1}{n}} $$

Explicação da perplexidade

In [220]:
"""Conforme os slides passados em aula, a perplexidade serve para avaliação de um language model, a alto nível, a perplexidade indica o quão 'surpreso' o modelo está em relação a uma sequência de tokens. A perplexidade é uma medida de incerteza, onde quanto menor a perplexidade, melhor o modelo, visto que o modelo está mais confiante em suas predições, e consequentemente, a sequência de tokens é mais provável de acordo com o modelo. 

    Matematicamente a intuição acima faz sentido, se enxergarmos a saída da softmax como probabilidades, e o calculo da perplexidade como sendo a raiz n-ésima do produtorio de 1 dividido pela probabilidade do token dado os tokens anteriores para cada token na sequência, temos que se a probabilidade daquele token for baixa, 1 / por um número bem pequeno resulta em um número muito grande, por outro lado, 1 dividido por um número próximo de 1 resulta em um número próximo de 1, assim, para cada token, quanto maior a probabilidade que o modelo dá para aquele token de fato menor a perplexidade.
    
    Na implementação abaixo, seria interessante obter probabilidades, como o modelo retorna o Log da softmax, iremos realizar a operação inversa (exponenciação) para voltar para o resultado da softmax, que irá nos gerar uma distribuição de probabilidade, assim, fazendo mais sentido para o cálculo da perplexidade.
    
    Outro ponti é que adicionamos um pequeno valor a probabilidade de cada token, isso por que é possivel (embora não seja muito provável) que um token tenha probabilidade 0 ou algo muito próximo disso, podendo causar uma divisão por 0."""

def perplexidade(modelo, texto):

    modelo.eval()

    hidden = modelo.initHidden()
    texto = preprocess_data(texto)
    texto = torch.tensor([vocab[i] for i in list(texto)], dtype=torch.long)
    
    perplexidade = 1
    produtorio_probs = 1
    for i in range(len(texto) - 1): #Sempre usamos os n ultimos tokens para predizer o atual
        token_atual = texto[i]
        proximo_token = texto[i+1]

        one_hot = torch.zeros(1, vocab_len)
        one_hot[0][token_atual] = 1

        y_pred, hidden = modelo.forward(one_hot, hidden)
        y_pred = torch.exp(y_pred) # Transformar log_softmax em softmax

        p_token = (y_pred[0][proximo_token]).item() #pegar probabilidade do token correto (proximo token)
        produtorio_probs *= (1 / (p_token + + 1e-12)) # Adicionando um valor pequeno para evitar divisão por 0 (mesmo que altamente improvável)

    n = len(texto)
    if n > 0:
        perplexidade = produtorio_probs ** (1 / float(n)) # Tirando a raiz enesima

    return perplexidade

In [221]:
perplexidade(rnn, 'when')

In [222]:
"""Como vamos plotar a perplexidade junto com a loss, não faz sentido avaliar ela no conjunto de treino, pois a mesma irá se comportar de maneira muito similar a loss, visto que a perplexidade é uma medida de incerteza, e a loss é uma medida de erro, váriaveis altamente correlacionadas, assim, vamos avaliar a perplexidade no conjunto de teste, para medir a qualidade do modelo."""

shakespeare_data = load_dataset('shakespeare-data.txt')
shakespeare_data = preprocess_data(shakespeare_data)

In [223]:
n_hidden = 64
rnn = RNN(vocab_len, n_hidden, vocab_len)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

In [224]:
'''Vamos usar a perplexidade como comparativo para os modelos'''

models_perplexidade = {}

In [225]:
"""Para o cáclulo da perplexidade, vamos adotar sequencias com o mesmo tamanho do max_sequence_len, visto que o modelo foi treinado com sequencias de tamanho fixo, e assim, a perplexidade será calculada para cada sequencia de tamanho max_sequence_len. Para exibição, faremos a média da perplexidade ao longo da época."""

perplex = []
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    perplexidade_total = 0
    count = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss
        
        perplexidade_total += perplexidade(rnn, shakespeare_data[start_idx: start_idx + max_sequence_len])
        count +=1
    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        print(f'Perplexidade nos dados de teste: {(perplexidade_total / count):.4f}')
        perplex.append(perplexidade_total / count)
        generate_rnn(rnn, prompt='when', tamanho_resposta=100)
        
models_perplexidade['RNN sem alteração'] = perplex

#### Atividade 8

**Alteração 1** - Adição de embeddings para representação de um caracter

In [226]:
'''Adicionando embeddings'''

embedding_dim = 128

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim, dropout_prob=0.5):
        super(RNN, self).__init__()
        self.embeddings = nn.Embedding(input_size, embedding_dim)
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout_prob)


    def forward(self, input, hidden):
        index = torch.nonzero(input).squeeze()
        index = index[-1]
        index = torch.tensor(index.item())

        embs = self.embeddings(index)
        embs = embs.unsqueeze(0)

        input_combined = torch.cat((embs, hidden), 1)
        hidden = self.i2h(input_combined)
        hidden = F.tanh(hidden)

        hidden = self.dropout(hidden)
        
        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [227]:
rnn = RNN(vocab_len, n_hidden, vocab_len, embedding_dim)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

n_hidden = 128
learning_rate = 0.0005
n_epochs = 5
print_every = 1
max_sequence_len = 25
rnn

In [228]:
perplex = []
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    perplexidade_total = 0
    count = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

        perplexidade_total += perplexidade(rnn, shakespeare_data[start_idx: start_idx + max_sequence_len])
        count +=1
    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        print(f'Perplexidade nos dados de teste: {(perplexidade_total / count):.4f}')
        perplex.append(perplexidade_total / count)
        generate_rnn(rnn, prompt='when', tamanho_resposta=100)

models_perplexidade['RNN alteração 1'] = perplex

In [229]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100)

In [230]:
generate_rnn(rnn, prompt='when', tamanho_resposta=100, aleatorio=True)

**ALteração 2**: Inicialização de pesos:
    Em sequencias muito longas, o gradiente pode explodir ou desaparecer, isso agravado em uma rede recorrente, justamente por que em cada step de tempo temos a mesma rede compartilhando os mesmos pesos, e assim diminuindo o grau de liberdade do modelo em comparação a uma mlp de multiplas camadas, assim, é mais "fácil" corrigir no inicio da sequencia, causando o problema de dependencia de longo prazo das redes neurais recorrentes vanilla. Uma das formas de tentar mitigar a explosão/ desaparecimento de gradiente nessas redes é com uma inicialização adequada dos pesos. A intuição é que a inicialização adequada normaliza a variância dos pesos, e assim, evitando uma alta amplitude.

In [307]:
embedding_dim = 128

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim, dropout_prob=0.5):
        super(RNN, self).__init__()
        self.embeddings = nn.Embedding(input_size, embedding_dim)
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout_prob)

        self.init_weights()


    def forward(self, input, hidden):
        index = torch.nonzero(input).squeeze()
        index = index[-1]
        index = torch.tensor(index.item())

        embs = self.embeddings(index)
        embs = embs.unsqueeze(0)

        input_combined = torch.cat((embs, hidden), 1)
        hidden = self.i2h(input_combined)
        hidden = F.tanh(hidden)

        hidden = self.dropout(hidden)

        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden
    
    

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

    def init_weights(self):
        nn.init.xavier_uniform_(self.i2h.weight) # Inicialização Xavier para as camadas lineares
        nn.init.xavier_uniform_(self.i2o.weight)
        nn.init.zeros_(self.i2h.bias)
        nn.init.zeros_(self.i2o.bias)

        nn.init.uniform_(self.embeddings.weight, -1.0, 1.0)

In [283]:
rnn = RNN(vocab_len, n_hidden, vocab_len, embedding_dim)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

n_hidden = 128
learning_rate = 0.0005
n_epochs = 5
print_every = 1
max_sequence_len = 20
rnn

In [233]:
perplex = []
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    perplexidade_total = 0
    count = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

        perplexidade_total += perplexidade(rnn, shakespeare_data[start_idx: start_idx + max_sequence_len])
        count +=1
    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        print(f'Perplexidade nos dados de teste: {(perplexidade_total / count):.4f}')
        perplex.append(perplexidade_total / count)
        generate_rnn(rnn, prompt='when', tamanho_resposta=100)

models_perplexidade['RNN alteração 2'] = perplex

**Alteração 3** Além disso, um grande limitante do modelo é a maneira a qual dividimos os dados, visto que definimos um tamanho fixo de sequência e amostramos ao longo do texto, sem que nos importemos com a qualidade dessas amostras. Assim, é possível que o modelo esteja sendo treinado em cima de uma sequencia que não faz sentido (uma palavra cortada por exemplo). Por isso, implementaremos um método de amostragem mais inteligente e que façam sentido, ou seja, que não cortem palavras.

In [289]:
vocab[' ']

In [302]:
n_hidden = 128
embedding_dim = 128
rnn = RNN(vocab_len, n_hidden, vocab_len, embedding_dim)

learning_rate = 0.0005
n_epochs = 5
print_every = 1
max_sequence_len = 25

criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

rnn

In [303]:
'''Alteramos a lógica anterior para que a sequencia passada obrigatoriamente contenha palavras inteiras completas, e não corte palavras no meio. Isso foi feito alterando omínimo de código possível, só alteramos os indices de inicio e fim de uma sequência '''
perplex = []

for epoch in range(1, n_epochs + 1):
    total_loss = 0
    perplexidade_total = 0
    count = 0
    start_idx = 0
    end_idx = max_sequence_len
    
    while start_idx < len(test_sentence) - max_sequence_len:
        
        while test_sentence[start_idx] != vocab[' ']:
            start_idx += 1

        while test_sentence[end_idx] != vocab[' '] and end_idx < len(test_sentence) - max_sequence_len:
            end_idx -= 1
            
        start_idx += 1 # Pegar o primeiro token após o espaço
        end_idx -= 1
        
        if start_idx >= end_idx:
            '''Sequência com única palavra'''
            continue

        
        input_tensor, target_tensor = create_input(test_sentence, start_idx, end_idx - start_idx) 
        loss = train(input_tensor, target_tensor)
        total_loss += loss
        
        
        perplexidade_total += perplexidade(rnn, shakespeare_data[start_idx: start_idx + max_sequence_len])
        count +=1
        
        start_idx = end_idx
        end_idx += max_sequence_len 

        
    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        print(f'Perplexidade média nos dados de teste: {(perplexidade_total / (count)):.4f}')
        perplex.append(perplexidade_total / count)
        generate_rnn(rnn, prompt='when', tamanho_resposta=100)
        print('\n')

models_perplexidade['RNN alteração 3'] = perplex

**Alteração 4** Adicionamos um schedular da learning_rate, de modo que ela decresce de acordo com o tempo (a cada época, conforme setado), além disso, alteramos a lógica do hidden no treinamento, fazendo o seguinte: Ao invés de a cada sequência iniciarmos o hidden zerado, iniciamos como o hidden da sequencia anterior, e cortamos o caminho do gradiente, para evitar uma dependencia de longo prazo. Além disso, ainda na ideia dos problemas de estabilidade das RNNS vanilla, adicionou-se a layer normalization, onde normalizamos as ativações de cada neurônio dentro de uma amostra específica, com isso, garantindo que as ativações de cada camada estejam dentro de uma faixa controlada

In [342]:
embedding_dim = 128

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim, dropout_prob=0.5):
        super(RNN, self).__init__()
        self.embeddings = nn.Embedding(input_size, embedding_dim)
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout_prob)

        self.layer_norm = nn.LayerNorm(hidden_size)

        self.init_weights()


    def forward(self, input, hidden):
        index = torch.nonzero(input).squeeze()
        index = index[-1]
        index = torch.tensor(index.item())

        embs = self.embeddings(index)
        embs = embs.unsqueeze(0)

        input_combined = torch.cat((embs, hidden), 1)
        hidden = self.i2h(input_combined)
        hidden = F.tanh(hidden)
        hidden = self.layer_norm(hidden)
        hidden = self.dropout(hidden)

        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden



    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

    def init_weights(self):
        nn.init.xavier_uniform_(self.i2h.weight) # Inicialização Xavier para as camadas lineares
        nn.init.xavier_uniform_(self.i2o.weight)
        nn.init.zeros_(self.i2h.bias)
        nn.init.zeros_(self.i2o.bias)

        nn.init.uniform_(self.embeddings.weight, -1.0, 1.0)

In [343]:
n_hidden = 128
embedding_dim = 128
rnn = RNN(vocab_len, n_hidden, vocab_len, embedding_dim)

learning_rate = 0.001
n_epochs = 5
print_every = 1
max_sequence_len = 25

criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

rnn

In [344]:
def train(input_tensor, target_tensor, hidden):

    rnn.zero_grad()
    loss = 0
    for i in range(input_tensor.size(0)):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, target_tensor[i].unsqueeze(0))

    torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=5) #clipping de gradiente para evitar o exploding gradient
    loss.backward()
    optimizer.step()

    return loss.item() / input_tensor.size(0), hidden.detach() #Retirar o rastreamento do hidden para a próxima iteração

In [345]:
step_size = 1  # a cada época, reduzir a learning rate
gamma = 0.9
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [346]:
perplex = []
for epoch in range(1, n_epochs + 1):
    hidden = rnn.initHidden()
    total_loss = 0
    perplexidade_total = 0
    count = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, fator_desconto*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss, hidden = train(input_tensor, target_tensor, hidden)
        total_loss += loss
        perplexidade_total += perplexidade(rnn, shakespeare_data[start_idx: start_idx + max_sequence_len])
        count +=1

        
        
    scheduler.step()

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
    print(f'Perplexidade nos dados de teste: {(perplexidade_total / count):.4f}')
    perplex.append(perplexidade_total / count)
    generate_rnn(rnn, prompt='when', tamanho_resposta=100)

models_perplexidade['RNN alteração 4'] = perplex

#### Comparativo

In [347]:
plt.figure(figsize=(10, 6))

# Lista de estilos de linha e cores para variedade
estilos = ['-', '--', '-.', ':']
cores = ['blue', 'green', 'red', 'orange', 'purple']

for i, (label, valores) in enumerate(models_perplexidade.items()):
    x = list(range(1, len(valores) + 1))
    plt.plot(x, valores,
             marker='o',
             linestyle=estilos[i % len(estilos)],
             color=cores[i % len(cores)],
             label=label)

plt.title('Perplexidade ao longo do Tempo')
plt.xlabel('Tempo')
plt.ylabel('Perplexidade')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Com isso, percebemos que as alterações melhoraram quando avaliado a perplexidade em cima de dados nunca vistos pelo modelo. A não ser pela alteração 3, que inclui uma sequencia de treinamento com palavras completas. Na alteração 3, como o tamanho de sequência varia, isso pode ter interferido diretamente na perplexidade.

#### Desafio

As Redes Neurais Recorrentes (RNNs) são projetadas para trabalhar com dados sequenciais, onde cada saída depende das entradas anteriores e do estado oculto anterior. Apesar de, em teoria, conseguirem capturar dependências de longo prazo, na prática elas enfrentam problemas de desaparecimento ou explosão de gradientes durante o treinamento. Isso dificulta o aprendizado em sequências longas.

Para superar esses desafios, surgiram as LSTMs (Long Short-Term Memory Networks) . Eles introduzem um mecanismo de memória mais inteligente que ajuda a preservar informações importantes ao longo do tempo. Aqui vai um resumo de como elas funcionam:

Estado de Célula : Pensa nele como uma "esteira" que carrega informações relevantes ao longo do tempo, permitindo que dados importantes fluam sem muitas interferências.

Três portas principais que controlam o fluxo de informações:

Porta de Esquecimento : Decida se o estado da célula anterior deve ser esquecido. Ela analisa o estado oculto anterior e a entrada atual para determinar isso.
Porta de Entrada : Determina quais novas informações serão selecionadas ao estado da célula. Ela cria um vetor de possíveis novas informações e decide o que realmente vai entrar.
Porta de Saída ( output gate ): Decida qual estado da célula atual será usado para produzir a saída naquele momento.
Essas portas usam funções de ativação específicas:

Função Sigmoide : Usada nas portas para gerar valores entre 0 e 1, ajudando a controlar de forma proporcional o que deve ser desligado ou esquecido.
Função Tangente Hiperbólica : Utilizada para criar o vetor de novas informações que podem ser adicionadas ao estado da célula, mantendo os valores entre -1 e 1.
Analogias com Ligações Residuais :

As portas nas LSTMs são semelhantes às conexões residuais em redes neurais profundas (como as ResNets). Ambas permitem que informações "saltem" etapas ou camadas, facilitando o fluxo de dados importantes e ajudando a evitar problemas com gradientes durante o treinamento.


In [429]:
'''Como temos um modelo word-level, nosso tokenizador deve quebrar o texto em palavras, e não em caracteres, assim, a entrada do modelo será uma sequência de palavras, e não de caracteres. Além disso, a saída do modelo será uma palavra, e não um caracter, assim, a saída do modelo será uma distribuição de probabilidade sobre o vocabulário, onde cada palavra terá uma probabilidade associada, e a palavra com maior probabilidade será a palavra prevista.'''


def tokenize_data(corpus):
    palavras = corpus.split()  # Quebra o texto em palavras

    vocabulario = {palavra: i for i, palavra in enumerate(set(palavras))}
    tokens = [vocabulario[palavra] for palavra in palavras]

    return tokens, vocabulario


'''Como vou criar o código da LSTM serão utilizados embeddings para representar uma palavra, com isso, a função create_input será alterada para retornar tensores com a representação númerica do token.'''

def create_input(tokens, start_idx, max_sequence_len):
    input = torch.tensor(tokens[start_idx: start_idx + max_sequence_len], dtype=torch.long) # Cria o tensor de entrada com tokens a partir do índice inicial, com comprimento máximo de sequência

    target = torch.tensor(tokens[start_idx + 1: start_idx + max_sequence_len + 1], dtype=torch.long) # Cria o tensor de target que é o próximo token em relação à sequência de entrada

    return input, target

'''FUNÇÃO PREPROCESS_DATA'''
def preprocess_data(corpus):
    corpus = unicodedata.normalize('NFD', corpus)  # Remove acentuação
    corpus = re.sub(r'\s+', ' ', corpus)  # Substitui quebras de linha e espaços múltiplos por um único espaço
    corpus = corpus.lower()  # Converte para minúsculas
    corpus = re.sub(r'[^a-zA-Z0-9.,;:!?\'" -]', '', corpus)  # Remove tudo exceto letras, números, pontuações e espaços

    #corpus = corpus.replace(';', '')
    corpus = corpus.replace('--', '')
    #corpus = corpus.replace("'", '')
    return corpus

In [430]:
test_sentence = load_dataset('obama-data.txt')

test_sentence = preprocess_data(test_sentence)

test_sentence = augment_text_by_word_freq(test_sentence, contagem_palavras, prob_map, stopword_prob=0.98)

test_sentence, vocab = tokenize_data(test_sentence)

vocab_len = len(vocab)
vocab_ix_to_word = {v: k for k, v in vocab.items()}

In [437]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim, dropout_prob=0.3):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        
        self.embeddings = nn.Embedding(input_size, embedding_dim) # Camada de embeddings que representarão cada token/palavra

        self.input_gate = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.forget_gate = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.output_gate = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.cell_candidate = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, hidden_size)


        # Camada de saída
        self.hidden2output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell_state):
        input = input.view(1)
        embedded = self.embeddings(input)

        combined = torch.cat((embedded, hidden), 1)

        # Calcular as portas
        i_t = torch.sigmoid(self.input_gate(combined))       # Porta de entrada
        f_t = torch.sigmoid(self.forget_gate(combined))      # Porta de esquecimento
        o_t = torch.sigmoid(self.output_gate(combined))      # Porta de saída
        g_t = torch.tanh(self.cell_candidate(combined))      # Candidato a novo estado de célula

        cell_state = f_t * cell_state + i_t * g_t # Atualizar o estado da célula

        hidden = o_t * torch.tanh(cell_state) # Atualizar o estado oculto
        
        hidden = self.linear(hidden)
    
        hidden = F.tanh(hidden)
        
        output = self.hidden2output(hidden)
        output = self.softmax(output)

        return output, hidden, cell_state

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

    def initCellState(self):
        return torch.zeros(1, self.hidden_size)

In [438]:
input_size = vocab_len
hidden_size = 64
output_size = vocab_len
embedding_dim = 32
n_epochs = 5
print_every = 1
max_sequence_len = 35
learning_rate=0.0005
lstm = LSTM(input_size, hidden_size, output_size, embedding_dim)
criterion = nn.NLLLoss()
optimizer = optim.SGD(lstm.parameters(), lr=learning_rate)

In [439]:
lstm

In [440]:
def generate_lstm(lstm, prompt=' ', tamanho_resposta=100):
    lstm.eval()
    oculto = lstm.initHidden()
    estado_celula = lstm.initCellState()
    prompt_processado = preprocess_data(prompt).split() 


    prompt_indices = []
    for word in prompt_processado:
        prompt_indices.append(vocab[word])

    prompt_tensor = torch.tensor(prompt_indices, dtype=torch.long)

    for token in prompt_tensor:
        token_input = token.view(1)
        output, oculto, estado_celula = lstm(token_input, oculto, estado_celula)

    y_pred = output

    # Geração do texto de resposta
    for _ in range(tamanho_resposta):
        token_previsto = torch.argmax(y_pred, dim=1).item()
        print(vocab_ix_to_word[token_previsto], end=' ')
        token_input = torch.tensor([token_previsto], dtype=torch.long)
        y_pred, oculto, estado_celula = lstm(token_input, oculto, estado_celula)


In [441]:
def train(input_tensor, target_tensor):
    hidden = lstm.initHidden().detach()
    cell_state = lstm.initCellState().detach()
    lstm.zero_grad()
    loss = 0
    for i in range(input_tensor.size(0)):
        output, hidden, cell_state = lstm(input_tensor[i], hidden, cell_state)
        loss += criterion(output, target_tensor[i].unsqueeze(0))
    loss.backward()
    optimizer.step()
    return loss.item() / input_tensor.size(0)

In [442]:
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in tqdm(range(0, len(test_sentence) - max_sequence_len, 50*max_sequence_len)):
        input_tensor, target_tensor = create_input(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {total_loss}')
        generate_lstm(lstm, 'badly', 30)

In [ ]:
generate_lstm(lstm, 'separatists', 30)